In [62]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from pyspark.sql.functions import date_format
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


In [4]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/10/03 22:24:17 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/03 22:24:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 22:24:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/03 22:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/03 22:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
# Read in data from ETL.py file
%run '../scripts/ETL.py' '../scripts/paths.json'
final_join3.limit(5)

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/03 22:24:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/03 22:25:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021
Egestas Nunc Asso...,11121775571,digital goods: bo...,6.58,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,11121775571,11.28829564583802,2bd2a61d-72e5-42d...,2021-08-20,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Morbi Accumsan In...,19618998054,tent and aWning s...,1.52,c,Christopher Rodri...,30554 Evans Strea...,NSW,Male,19618998054,62.90176609196828,3582b1f8-4577-403...,2021-05-16,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Dolor Egestas PC,94472466107,"cable, satellite,...",6.23,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,94472466107,172.15375126873164,cb05d49f-c2fa-453...,2021-07-22,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Urna Justo Indust...,31472801314,music shops - mus...,6.56,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,31472801314,0.4894787650356477,aeec15c1-67e8-4cb...,2021-05-18,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Sem Pellentesq...,35424691626,"computers, comput...",3.9,b,Christopher Rodri...,30554 Evans Strea...,NSW,Male,35424691626,7.360217018778133,9df473ba-102d-461...,2021-07-04,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694


take rate
state
gender
year
month
day
postcode
merchant name



In [28]:
final_join3.createOrReplaceTempView("temp")

d = spark.sql(""" 

SELECT COUNT(DISTINCT take_rate), merchant_name
FROM temp
GROUP BY merchant_name
""")

d.show()

+-------------------------+--------------------+
|count(DISTINCT take_rate)|       merchant_name|
+-------------------------+--------------------+
|                        1|     Erat Semper Ltd|
|                        1|Dictum Mi Incorpo...|
|                        1|At Augue Corporation|
|                        1|Porttitor Eros In...|
|                        1|Mollis Integer Co...|
|                        1|Varius Orci Insti...|
|                        1|Volutpat Nulla In...|
|                        1|      Amet Nulla Ltd|
|                        1|Hendrerit Consect...|
|                        1|   Malesuada Vel Ltd|
|                        1|  Vel Turpis Company|
|                        1|Elit Sed Consequa...|
|                        1|        Malesuada PC|
|                        1|   Pede Praesent Ltd|
|                        1|   Dictum Mi Limited|
|                        1|Vulputate Velit E...|
|                        1| Phasellus Nulla LLC|
|                   

In [25]:
merchants = spark.read.parquet("/Users/Kasturi/Documents/GitHub/generic-buy-now-pay-later-project-group-10-bnpl/data/tables/tbl_merchants.parquet")

In [26]:
merchants.show(5)

+--------------------+--------------------+------------+
|                name|                tags|merchant_abn|
+--------------------+--------------------+------------+
|       Felis Limited|((furniture, home...| 10023283211|
|Arcu Ac Orci Corp...|([cable, satellit...| 10142254217|
|    Nunc Sed Company|([jewelry, watch,...| 10165489824|
|Ultricies Digniss...|([wAtch, clock, a...| 10187291046|
| Enim Condimentum PC|([music shops - m...| 10192359162|
+--------------------+--------------------+------------+
only showing top 5 rows



In [6]:
final_join3.count()

10540181

In [46]:
tagged_merchants = pd.read_csv("../data/curated/tagged_merchants.csv")

In [47]:
tagged_merchants = tagged_merchants.iloc[:,1:]

In [48]:
tagged_merchants.drop(['tags', 'name', 'cleaned_tags', 'store_type'], axis=1, inplace=True)

In [49]:
tagged_merchants.to_parquet("../data/curated/tagged_merchants.parquet")

In [50]:
tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

In [51]:
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)

In [52]:
tagged_merchants_sdf.show(5)

+-------------------+--------------------+
|tagged_merchant_abn|            category|
+-------------------+--------------------+
|        10023283211|           Furniture|
|        10142254217|         Electronics|
|        10165489824|        Toys and DIY|
|        10187291046|        Toys and DIY|
|        10192359162|Books, Stationary...|
+-------------------+--------------------+
only showing top 5 rows



In [53]:
final_join3.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
LEFT JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")



In [54]:
joint = joint.drop('tagged_merchant_abn')

In [55]:
joint.count()

10540181

In [56]:
joint.createOrReplaceTempView("group")

a = spark.sql(""" 

SELECT *, (dollar_value - take_rate) AS total_earning
FROM group
""")

In [57]:
a.createOrReplaceTempView("group_earnings")

b = spark.sql(""" 

SELECT merchant_name, SUM(total_earning) AS total_revenue, COUNT(merchant_abn) AS no_of_transactions
FROM group_earnings
GROUP BY merchant_name
""")

b.show()

+--------------------+------------------+------------------+
|       merchant_name|     total_revenue|no_of_transactions|
+--------------------+------------------+------------------+
|   Dictum Mi Limited| 897687.9661494899|              9880|
|Volutpat Nulla In...| 836578.8864434501|             10561|
|Mollis Integer Co...|384905.39521080774|             12234|
|Hendrerit Consect...|239274.53116224083|              3010|
|Donec Luctus Indu...|  1063577.03533018|              3113|
|Varius Orci Insti...|147376.27431885863|             19771|
|Elit Sed Consequa...| 5724004.642736054|              9689|
|Dictum Mi Incorpo...| 775536.8639499726|              1236|
|At Augue Corporation|125257.92718248101|              1544|
|      Montes Limited|429378.17366086063|               673|
|     Erat Semper Ltd| 639325.5782926289|              1781|
| Semper Incorporated| 92248.05999937073|               926|
|Porttitor Eros In...|121304.87964357901|              1262|
|   Malesuada Vel Ltd| 1

In [59]:
# Extracting the year, month, day from the timestamp

a = a.withColumn("Year", 
date_format('order_datetime', 'yyyy'))

a  = a.withColumn("Month", 
date_format('order_datetime', 'MMMM'))


a = a.withColumn("Day",
date_format(("order_datetime"), "E"))


In [60]:
a.show(5)

+--------------------+------------+--------------------+---------+--------------+--------------------+--------------------+-----+------+------------------+------------------+--------------------+--------------+-------+-----------+---------+---------+---------+-------------------+----------------+-----------+-------------+-------------+----------+---------------+---------------+---------------+--------------------+------------------+----+------+------------+
|       merchant_name|merchant_abn|          categories|take_rate|revenue_levels|                name|             address|state|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|postcodes|  int_sa2| SA2_code|           SA2_name|income_2018-2019|total_males|total_females|total_persons|state_code|     state_name|population_2020|population_2021|            category|     total_earning|Year| Month|week_day_abb|
+--------------------+------------+--------------------+---------+----------

In [61]:
a.createOrReplaceTempView("temp")

c = spark.sql(""" 

SELECT *
FROM temp
GROUP BY merchant_name
""")

c.limit(5)

AnalysisException: expression 'temp.merchant_abn' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [merchant_name#193], [merchant_name#193, merchant_abn#31L, categories#170, take_rate#175, revenue_levels#181, name#17, address#18, state#19, gender#21, trans_merchant_abn#214L, dollar_value#41, order_id#42, order_datetime#43, user_id#35L, consumer_id#36L, postcodes#315, int_sa2#499, SA2_code#535L, SA2_name#536, income_2018-2019#537L, total_males#538L, total_females#539L, total_persons#540L, state_code#541L, ... 8 more fields]
+- SubqueryAlias temp
   +- View (`temp`, [merchant_name#193,merchant_abn#31L,categories#170,take_rate#175,revenue_levels#181,name#17,address#18,state#19,gender#21,trans_merchant_abn#214L,dollar_value#41,order_id#42,order_datetime#43,user_id#35L,consumer_id#36L,postcodes#315,int_sa2#499,SA2_code#535L,SA2_name#536,income_2018-2019#537L,total_males#